# Módulo 07, Tarefa 01

Vamos começar a mexer na nossa base de projeto? Já fizemos um exercício de montar a variável resposta, já pudemos perceber que essa atividade pode não ser trivial. Vamos agora trabalhar a base para que fique propícia ao *scikitlearn* para trabalharmos.

Lembrando, a base se chama demo01.csv, e originalmente está publicada [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).

#### 1) Carregue a base e avalie:

- As variáveis
- Tipos de dados de cada variável
- Quantidade de missings
- Distribuição da variável resposta (mau)

In [1]:
import pandas as pd

file_path = 'demo01.csv'
demo_data = pd.read_csv(file_path)

print(demo_data.head())


  sexo posse_de_veiculo posse_de_imovel  qtd_filhos            tipo_renda  \
0    M                Y               Y           0               Working   
1    F                N               Y           0  Commercial associate   
2    F                N               Y           0  Commercial associate   
3    M                Y               Y           0               Working   
4    F                Y               N           0               Working   

                        educacao          estado_civil    tipo_residencia  \
0  Secondary / secondary special               Married  House / apartment   
1  Secondary / secondary special  Single / not married  House / apartment   
2  Secondary / secondary special  Single / not married  House / apartment   
3               Higher education               Married  House / apartment   
4              Incomplete higher               Married  House / apartment   

       idade  tempo_emprego  possui_celular  possui_fone_comercial  \
0  5

#### 2) Vamos montar um metadados

1. Crie um dataframe com os nomes de cada variável e o tipo de dados de cada variável.
2. Adicione uma coluna nesse *dataframe* chamada "qtd_categorias" e coloque nela o número de categorias correspondente de cada variável.
    Dica:
        1. inicie uma lista vazia
        2. faça um for ao longo dos nomes das variáveis,
        3. conte o número de categorias dessa variável
        4. acumule essa informação de 3. na lista que você criou em 1.
        5. No final, essa lista pode ser atribuída à nossa variável.

In [7]:
nomes_variaveis = []
tipos_dados = []
qtd_categorias = []

for coluna in demo_data.columns:
    nomes_variaveis.append(coluna)  
    tipos_dados.append(demo_data[coluna].dtype)  
    
    if isinstance(demo_data[coluna].dtype, pd.CategoricalDtype) or pd.api.types.is_object_dtype(demo_data[coluna]):
        qtd_categorias.append(demo_data[coluna].nunique()) 
    else:
        qtd_categorias.append(None) 

metadados = pd.DataFrame({
    'Nome': nomes_variaveis,
    'Tipo de Dados': tipos_dados,
    'Qtd Categorias': qtd_categorias
})

print(metadados)


                     Nome Tipo de Dados  Qtd Categorias
0                    sexo        object             2.0
1        posse_de_veiculo        object             2.0
2         posse_de_imovel        object             2.0
3              qtd_filhos         int64             NaN
4              tipo_renda        object             5.0
5                educacao        object             5.0
6            estado_civil        object             5.0
7         tipo_residencia        object             6.0
8                   idade       float64             NaN
9           tempo_emprego       float64             NaN
10         possui_celular         int64             NaN
11  possui_fone_comercial         int64             NaN
12            possui_fone         int64             NaN
13           possui_email         int64             NaN
14  qt_pessoas_residencia       float64             NaN
15                    mau          bool             NaN


#### 3) Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}. Crie um *dataframe* apenas com as variáveis apropriadas para entrada no scikitlearn - elimine as variáveis tipo *str*, mantendo apenas suas versões *dummy*.

In [11]:
variaveis_qualitativas = demo_data.select_dtypes(include=['object', 'category'])

variaveis_dummies = pd.get_dummies(variaveis_qualitativas)

variaveis_numericas = demo_data.select_dtypes(include=['number'])

df_final = pd.concat([variaveis_numericas, variaveis_dummies], axis=1)

print(df_final.head())


   qtd_filhos      idade  tempo_emprego  possui_celular  \
0           0  58.832877       3.106849               1   
1           0  52.356164       8.358904               1   
2           0  52.356164       8.358904               1   
3           0  46.224658       2.106849               1   
4           0  29.230137       3.021918               1   

   possui_fone_comercial  possui_fone  possui_email  qt_pessoas_residencia  \
0                      0            0             0                    2.0   
1                      0            1             1                    1.0   
2                      0            1             1                    1.0   
3                      1            1             1                    2.0   
4                      0            0             0                    2.0   

   sexo_F  sexo_M  ...  estado_civil_Married  estado_civil_Separated  \
0   False    True  ...                  True                   False   
1    True   False  ...          

#### 4) Qual variável é mais poderosa?

Considere as variáveis ```possui_email``` e ```posse_de_veiculo```. Faça uma tabela cruzada entre elas e responda qual delas te parece mais poderosa para prever a probabilidade de ```mau = 1```?

In [19]:
# Converter a variável 'mau' para 1 e 0
demo_data['mau'] = demo_data['mau'].astype(int)

# Converter 'possui_email' e 'posse_de_veiculo' para 1 e 0
demo_data['possui_email'] = demo_data['possui_email'].astype(int)
demo_data['posse_de_veiculo'] = demo_data['posse_de_veiculo'].map({'Y': 1, 'N': 0})

print("Categorias da variável 'mau':")
print(demo_data['mau'].value_counts())


Categorias da variável 'mau':
mau
0    16260
1      390
Name: count, dtype: int64


In [21]:
tabela_cruzada_email = pd.crosstab(demo_data['possui_email'], demo_data['mau'])
tabela_cruzada_veiculo = pd.crosstab(demo_data['posse_de_veiculo'], demo_data['mau'])

print("\nTabela Cruzada 'possui_email':")
print(tabela_cruzada_email)

print("\nTabela Cruzada 'posse_de_veiculo':")
print(tabela_cruzada_veiculo)


Tabela Cruzada 'possui_email':
mau               0    1
possui_email            
0             14810  360
1              1450   30

Tabela Cruzada 'posse_de_veiculo':
Empty DataFrame
Columns: []
Index: []


In [ ]:
proporcao_email = tabela_cruzada_email[1] / tabela_cruzada_email.sum(axis=1)
proporcao_veiculo = tabela_cruzada_veiculo[1] / tabela_cruzada_veiculo.sum(axis=1)

# Mostrar as proporções
print("\nProporções para 'possui_email':")
print(proporcao_email)

print("\nProporções para 'posse_de_veiculo':")
print(proporcao_veiculo)

# Comparar as proporções
print("\nComparação:")
if proporcao_email.max() > proporcao_veiculo.max():
    print("A variável 'possui_email' parece ser mais poderosa.")
else:
    print("A variável 'posse_de_veiculo' parece ser mais poderosa.")

#### 5) Salve a base, pois ela será utilizada no final deste módulo.